# Ticker Correlation Analyzer (Investpy)

Analyze the rolling correlation between two tickers. Use the controls below to select the tickers, date range, and the rolling window size (in days) for the correlation calculation.

In [ ]:
!pip install investpy
from __future__ import annotations

import pandas as pd
import investpy as inv
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

plt.style.use("seaborn-v0_8-whitegrid")
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
default_end = pd.Timestamp.today().normalize()
default_start = default_end - pd.DateOffset(years=10)

ticker1_input = widgets.Text(    value="QQQ",    description="Ticker 1",    placeholder="e.g. QQQ",    disabled=False,)

ticker2_input = widgets.Text(    value="GLD",    description="Ticker 2",    placeholder="e.g. GLD",    disabled=False,)

start_input = widgets.DatePicker(    description="Start Date",    value=default_start.to_pydatetime(),)

end_input = widgets.DatePicker(    description="End Date",    value=default_end.to_pydatetime(),)

window_input = widgets.IntText(    value=22,    description="Window (days)",    disabled=False)

run_button = widgets.Button(description="Analyze", button_style="primary")
output = widgets.Output()

def _to_timestamp(value):
    if value is None:
        return None
    return pd.Timestamp(value)

def analyze_correlation(_=None):
    with output:
        output.clear_output()
        ticker1 = ticker1_input.value.strip().upper()
        ticker2 = ticker2_input.value.strip().upper()
        start = _to_timestamp(start_input.value)
        end = _to_timestamp(end_input.value)
        window = window_input.value

        if not all([ticker1, ticker2]):
            print("Please enter two ticker symbols.")
            return
        if start is None or end is None:
            print("Please select both start and end dates.")
            return
        if start >= end:
            print("Start date must be before end date.")
            return
        if window <= 1:
            print("Window size must be greater than 1.")
            return

        try:
            # Fetch data for each ticker individually, which is more robust.
            history1 = inv.get_stock_historical_data(                stock=ticker1, country='United States', from_date=start.strftime('%d/%m/%Y'), to_date=end.strftime('%d/%m/%Y')
            )
            history2 = inv.get_stock_historical_data(                stock=ticker2, country='United States', from_date=start.strftime('%d/%m/%Y'), to_date=end.strftime('%d/%m/%Y')
            )

            if history1.empty or history2.empty:
                print(                    "Could not download complete data for one or both tickers. "
                    "Please check the symbols and date range."
                )
                return

            # Combine the adjusted 'Close' prices into a single DataFrame.
            data = pd.DataFrame({ticker1: history1["Close"], ticker2: history2["Close"]})
            data.dropna(inplace=True)  # Ensure we only work with overlapping dates.

            if data.empty:
                print("No overlapping price data found for the selected tickers and dates.")
                return

        except Exception as exc:
            print(f"Failed to download data: {exc}")
            return

        returns = data.pct_change()
        correlation = returns[ticker1].rolling(window=window).corr(returns[ticker2])

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

        # Plot prices
        ax1.plot(data.index, data[ticker1], label=f"{ticker1} Adj Close")
        ax1.plot(data.index, data[ticker2], label=f"{ticker2} Adj Close")
        ax1.set_title(f"{ticker1} vs {ticker2} Adjusted Close Price")
        ax1.set_ylabel("Price (USD)")
        ax1.legend()
        ax1.grid(True)

        # Plot correlation
        ax2.plot(correlation.index, correlation.values, label=f"Rolling Correlation", color='purple')
        ax2.set_title(f"{window}-Day Rolling Correlation")
        ax2.set_ylabel("Correlation Coefficient")
        ax2.set_xlabel("Date")
        ax2.axhline(0, color='grey', linestyle='--')
        ax2.legend()
        ax2.grid(True)

        fig.autofmt_xdate()
        plt.tight_layout()
        plt.show()

run_button.on_click(analyze_correlation)

ticker_box = widgets.HBox([ticker1_input, ticker2_input])
date_box = widgets.HBox([start_input, end_input])
controls = widgets.VBox([ticker_box, date_box, window_input, run_button])

display(controls, output)
